<a href="https://colab.research.google.com/github/alen-ka/githubTest/blob/main/Lab_3_Ex_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3 Exercise 3
## Binary Classification

We are given a dataset that consists of biological features of a person and whether or not they are a smoker.

There are *22 descriptive features* and *1 target column* in the dataset as described in the Table below. You are tasked with creating a predictive model to predict the column "SMK_stat_type_cd", which specifies smoker/non-smoker status of the person. You are expected to use classification methods that was shown in the previous exercises.



|      Column      |                                 Description                                 |
|:----------------:|:---------------------------------------------------------------------------:|
| identified_gender              | male, female                                                                |
| age              | round up to 5 years                                                         |
| height           | round up to 5 cm[cm]                                                        |
| weight           | [kg]                                                                        |
| sight_left       | eyesight(left)                                                              |
| sight_right      | eyesight(right)                                                             |
| hear_left        | hearing left, 1(normal), 2(abnormal)                                        |
| hear_right       | hearing right, 1(normal), 2(abnormal)                                       |
| SBP              | Systolic blood pressure[mmHg]                                               |
| DBP              | Diastolic blood pressure[mmHg]                                              |
| BLDS             | BLDS or FSG(fasting blood glucose)[mg/dL]                                   |
| tot_chole        | total cholesterol[mg/dL]                                                    |
| HDL_chole        | HDL cholesterol[mg/dL]                                                      |
| LDL_chole        | LDL cholesterol[mg/dL]                                                      |
| triglyceride     | triglyceride[mg/dL]                                                         |
| hemoglobin       | hemoglobin[g/dL]                                                            |
| urine_protein    | protein in urine, 1(-), 2(+/-), 3(+1), 4(+2), 5(+3), 6(+4)                  |
| serum_creatinine | serum(blood) creatinine[mg/dL]                                              |
| SGOT_AST         | SGOT(Glutamate-oxaloacetate transaminase) AST(Aspartate transaminase)[IU/L] |
| SGOT_ALT         | ALT(Alanine transaminase)[IU/L]                                             |
| gamma_GTP        | y-glutamyl transpeptidase[IU/L]                                             |
| SMK_stat_type_cd **(Target)** | Smoking state, 0(never), 1(active smoker)                      |


### The stages of this problem can be decomposed as follows:
1. Data Preparation
* Ensure data is in correct format (numerical and not string)
* Normalize the data for better convergence (optional)
* Split the data into train/test subsets

2. Model Selection
* Instanciate models and fit on training data
* Evaluate model performance on testing data
* Select model with best performance

3. Submit model
* Your model will be evaluated on data that is kept separate from training/testing data
* The predictions from your model will be uploaded to the course server where it will be evaluated

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/4c16-labs/code/lab-03/
!unzip -o data.zip
!echo 'data/*' > .gitignore

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/4c16-labs/code/lab-03
Archive:  data.zip
  inflating: data/data.csv           
  inflating: data/validation.csv     


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Data Preparation
# The data is supplied as ".csv" format
# There are 23 columns in csv file, with 22 columns being features and 1 column being the target

# The csv file can be read into as a dataframe using the pandas library
DataFrame = pd.read_csv('data/data.csv')

# Shuffle the data
DataFrame = DataFrame.sample(frac=1)

# Visualize the first 5 rows of the imported dataframe
display(DataFrame.head(6))

,identified_gender,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
18236,Female,50,160,50,67.0,1.2,0.9,1.0,1.0,120.0,...,63.0,75.0,65.0,13.8,1.0,0.8,21.0,11.0,20.0,0.0
1355,Male,35,175,70,78.0,1.2,1.2,1.0,1.0,108.0,...,60.0,101.0,83.0,16.0,1.0,0.8,20.0,17.0,18.0,0.0
22656,Female,45,150,55,76.0,0.9,1.0,1.0,1.0,124.0,...,47.0,78.0,232.0,13.5,1.0,0.6,42.0,48.0,27.0,0.0
6462,Female,40,155,60,83.0,1.2,1.2,1.0,1.0,100.0,...,42.0,74.0,57.0,12.5,1.0,0.5,16.0,17.0,16.0,0.0
35713,Male,80,165,50,74.0,1.0,1.0,1.0,2.0,116.0,...,62.0,44.0,75.0,12.9,1.0,0.7,25.0,26.0,26.0,1.0
36589,Female,50,155,70,89.0,0.8,0.8,1.0,1.0,130.0,...,45.0,147.0,196.0,14.6,1.0,0.7,21.0,25.0,23.0,0.0


In [ ]:
# The column "identified_gender" contains non-numeric data. This cannot be used as is and needs to be converted to numerical representations
# We can encode the identified gender to a numerical representation by letting "female" == 0, and "male" == 1.
# This can be done manually by using conditional statements on the dataframe, but luckily for us there is a simpler method
# We can use the built in LabelEncoder function of sklearn to do this

LabelEncoder = sklearn.preprocessing.LabelEncoder()
LabelEncoder.fit(DataFrame['identified_gender'])
DataFrame['identified_gender'] = LabelEncoder.transform(DataFrame['identified_gender'])

# Label encoder sorts the input column of data in alphabetical order, and then assigns a numerical value to each unique entry
# This results in 'female' being mapped to 0, 'male' being mapped to 1
display(DataFrame.head(6))

,identified_gender,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
18236,0,50,160,50,67.0,1.2,0.9,1.0,1.0,120.0,...,63.0,75.0,65.0,13.8,1.0,0.8,21.0,11.0,20.0,0.0
1355,1,35,175,70,78.0,1.2,1.2,1.0,1.0,108.0,...,60.0,101.0,83.0,16.0,1.0,0.8,20.0,17.0,18.0,0.0
22656,0,45,150,55,76.0,0.9,1.0,1.0,1.0,124.0,...,47.0,78.0,232.0,13.5,1.0,0.6,42.0,48.0,27.0,0.0
6462,0,40,155,60,83.0,1.2,1.2,1.0,1.0,100.0,...,42.0,74.0,57.0,12.5,1.0,0.5,16.0,17.0,16.0,0.0
35713,1,80,165,50,74.0,1.0,1.0,1.0,2.0,116.0,...,62.0,44.0,75.0,12.9,1.0,0.7,25.0,26.0,26.0,1.0
36589,0,50,155,70,89.0,0.8,0.8,1.0,1.0,130.0,...,45.0,147.0,196.0,14.6,1.0,0.7,21.0,25.0,23.0,0.0


In [ ]:
# We should now separate the input features from the target feature and store them as different variables
# We do this by slicing what columns of data we want from the dataframe
# Let's first see the columns available to us by printing DataFrame.columns
print(DataFrame.columns)

Index(['identified_gender', 'age', 'height', 'weight', 'waistline',
       'sight_left', 'sight_right', 'hear_left', 'hear_right', 'SBP', 'DBP',
       'BLDS', 'tot_chole', 'HDL_chole', 'LDL_chole', 'triglyceride',
       'hemoglobin', 'urine_protein', 'serum_creatinine', 'SGOT_AST',
       'SGOT_ALT', 'gamma_GTP', 'SMK_stat_type_cd'],
      dtype='object')


In [ ]:
# The column titled "SMK_stat_type_cd" is our target, and all remaining variables are our input features.
features_columns = DataFrame.columns[:-1]
target_column = DataFrame.columns[-1:]

DataFrame_X = DataFrame[features_columns]   # Selects only Input features
DataFrame_Y = DataFrame[target_column]      # Selects only Target variable

In [ ]:
# Split Data into Train/Test split
# Change the variable "test_frac" to reflect the percentage/fraction of test data in the resulting split
test_frac = 0.15

Train_X, Test_X, Train_Y, Test_Y = train_test_split(DataFrame_X, DataFrame_Y, test_size=test_frac)
print(f"Number of rows for Training:\t{Train_X.shape[0]}\nNumber of rows for Testing:\t{Test_X.shape[0]}")
print(Train_X.shape)
print(Train_Y.shape)

Number of rows for Training:	42500
Number of rows for Testing:	7500
(42500, 22)
(42500, 1)


In [ ]:
# EDIT THIS CELL
# Construct a dictionary of prediction models to compare
# Uncomment the below dictionary and insert as many prediction models as you like.
# You may have used binary classification models in previous exercises
# You may also have to import these modules/libraries to be able to use them
import sklearn.linear_model as skl_lm

from sklearn.datasets import fetch_openml
from sklearn.utils import check_random_state

from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression


PredictionModels = {
     #'Logistic Regression': LogisticRegression(),
     #'K-NN Classificaiton': code.for.KNN.classification.model.here(),
     #Logistic Regression': LogisticRegression(C=50.0 / 5000,
                                #multi_class='auto',
                                #solver='saga', tol=0.002),
    #'K-NN Classificaiton': KNeighborsClassifier(3),
    #'Linear SVM' : SVC(kernel="linear", C=0.025, probability=True),
    #'RBF SVM'    : SVC(C=5, gamma=0.05, probability=True),
    #'Decision Tree' : DecisionTreeClassifier(max_depth=5)
    'Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Logistic': LogisticRegression(solver='newton-cg',random_state = 11),
     #'Linear SVM' :       SVC(kernel="linear"),
     #'RBF SVM'    :       SVC(),
    'Decision Tree' :    DecisionTreeClassifier(max_depth=7)
 }

models = list(PredictionModels.keys())

print("Fitting models, this may take a while")
print(Train_X.shape)
print(Train_Y.shape)

for _model in models:
    print(_model)
    PredictionModels.get(_model).fit(Train_X,Train_Y)
    y_pred = PredictionModels.get(_model).predict(Test_X)
    f1 = f1_score(Test_Y, y_pred, average='macro')
    print(f1)


    # This loop goes through the models in the variable "PredictionModels"
    # Complete the below code block to fit to the model to training data "Train_X, Train_Y"
    # Peform predictions on the fitted model on the Train set and Test set
    # compute f1 score


Fitting models, this may take a while
(42500, 22)
(42500, 1)
Nearest Neighbors


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.7042130359426362
Logistic


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.792430795435494
Decision Tree
0.7897220401325882


In [ ]:
#from sklearn.metrics import accuracy_score
#from sklearn.metrics import recall_score
#from sklearn.metrics import precision_score
#from sklearn.metrics import f1_score


#for name, classifier in classifiers.items():
    #print('predictions for classifer: {}'.format(name))
   # start_time = time.time()
    #y_pred = classifier.predict(X_test)
    #end_time = time.time()

    #m = {}
   # m['f1'] = f1_score(y_test, y_pred, average='macro')
    #m['accuracy'] = accuracy_score(y_test, y_pred)
    #m['precision'] = precision_score(y_test, y_pred, average='macro')
    #m['recall'] = recall_score(y_test, y_pred, average='macro')
    #m['prediction_time'] = end_time - start_time
   # metrics[name].update(m)
#print("done")


In [ ]:
# Select the model with best f1 score and write down the
# corresponding 'key' value from the 'PredictionModels' variable
# Eg if Logistic Regression is chosen then: chosen_model = 'Logistic Regression'
chosen_model = 'Logistic Regression'

In [14]:
# DO NOT EDIT.
# Generate predictions using "chosen_model" and save to file

backend_features = pd.read_csv('data/validation.csv')
backend_features['identified_gender'] = LabelEncoder.transform(backend_features['identified_gender'])
backend_preds = PredictionModels[chosen_model].predict(backend_features)
np.savez_compressed('lab3_ex3_preds', lab3_model=backend_preds)

# Remember to push your changes to the git server for marking!

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
